##Library

In [ ]:
import copy
import heapq
import json
from timeit import default_timer as timer
import os
import psutil
import random
import numpy as np
import matplotlib.pyplot as plt

## Statistic data collect

In [ ]:

def get_process_memory():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

def memory_used():
  mem = psutil.virtual_memory()[2]
  return mem
def profile(store):
    def prof_stor(func):
        def wrapper(*args, **kwargs):
            mem_before = get_process_memory()
            visited = set()
            itr = [0,0]
            start = timer()
            try:
                result = func(*args, visited, itr, **kwargs)          
            except Exception as e:
                print(e)
                result = None
            elapsed_time = timer() - start
            mem_after = get_process_memory()
            
            if result and store:
                print('//---------------------// \nFinal ', end='')
                result.print_state()
                stats[store]= {
                    'elapsed_time' : elapsed_time,                 
                    #'memory' : mem_after - mem_before,
                    'iters' : itr[0],
                    'path_length': itr[1],
                    'cpu' : psutil.cpu_percent(4),
                    }
            else:
                print('No Soluton found !')
                stats[store]= {
                    'elapsed_time' : elapsed_time,
                    #'memory' : mem_after - mem_before,
                    'iters' : 0,
                    'path_length': 0,
                    'cpu' : psutil.cpu_percent(4),
                    }
            return result
        return wrapper
    return prof_stor

## Game

In [ ]:
#Function For Random Initial State Input
def random_Input(n): # n = number of row and column
  randInput = []
  testInput = []
  indexInput = 0
  while(len(randInput) < n*n):
    randnum = random.randint(1, n)      
    if randInput.count(randnum) < n :
      randInput.append(randnum)       
  
  for i in range(n):
    sliceRow = []
    for j in range(n):
      indexValue = randInput[indexInput]
      if(indexValue == 1):
        value = '🟥'
      elif(indexValue == 2):
        value = '🟨'
      elif(indexValue == 3):
        value = '🟦'
      elif(indexValue == 4):
        value = '🟩'
      elif(indexValue == 5):
        value = '⬜️'
        
      sliceRow.append(value)
      indexInput += 1 
    #sliceRow = str(sliceRow).replace('[','').replace(']','')
    testInput.append(sliceRow)

  return testInput
  #dis = ['🟥,🟥,🟥,🟥','🟨,🟨,🟨,🟨','🟦,🟦,🟦,🟦','🟩,🟩,🟩,🟩'] Example input

In [ ]:
stats = {}

class Board:
    def __init__(self, n: int):
        self.size = n
        self.state = []

    def __eq__(self, other):
        return self.h() == other.h()
    def __lt__(self, other):
        return self.h() < other.h()

    def is_goal_state(self):
        return self.h() == 0

    def h(self):
        dist = 0
        val = 0
        for i in range(0, self.size):
            for j in range(0, self.size):
                dir_x = [1,0,-1,0]
                dir_y = [0,1,0,-1]
                for k in range(0, 4):
                    new_x = i+dir_x[k]
                    new_y = j+dir_y[k]
                    if new_x >=0 and new_x <self.size and new_y >=0 and new_y < self.size:
                        if self.state[new_x][new_y] == self.state[i][j]:
                            dist+=1
        return dist

    def input_state(self,n):
        #dis = ['🟥,🟥,🟦,🟦','🟨,🟥,🟩,🟨','🟥,🟨,🟦,🟨','🟨,🟦,🟩,🟥']
        dis = random_Input(n) # Set 4 mean 4x4  if 5 is 5x5
        n = self.size
        print('Enter the colour Board state as {0}x{0} matrix, elements separated by a {1}:'.format(n, '","'))
        for i in dis:
            #row = i.split(',') #*******************Comment this if use Random Input***********************
            self.state.append(i)
        self.validate_input()

    def validate_input(self):
        vals = []
        for i in range(0, self.size):
            print(self.state[i])
            if len(self.state[i]) != self.size:
                raise AssertionError('Board specified is not square')
        for i in range(0, self.size):
            for j in range(0, self.size):
                if str(self.state[i][j]).strip() not in ["🟥","🟩","🟦","🟨","⬜️"]:
                    raise AssertionError('Invalid Colour')
                else:
                    self.state[i][j] = str(self.state[i][j]).strip()

    def print_state(self):
        n = self.size
        print('Board state: ')
        for i in range(0, n):
          for j in range(0, n):
            print(self.state[i][j], end = '')
          print('')

    def generate_states(self):
        n = self.size
        new_states = []
        for i in range(0, n):
            for j in range(0, n):
                dir_x = [1,0,-1,0]
                dir_y = [0,1,0,-1]
                bad = False
                for k in range(0, 4):
                    new_x = i+dir_x[k]
                    new_y = j+dir_y[k]
                    if new_x >=0 and new_x <self.size and new_y >=0 and new_y < self.size:
                        if self.state[new_x][new_y] == self.state[i][j]:
                            bad = True
                if not bad:
                    continue
                for k in range(0, 4):
                    new_x = i+dir_x[k]
                    new_y = j+dir_y[k]
                    if new_x >=0 and new_x <self.size and new_y >=0 and new_y < self.size:
                        if self.state[new_x][new_y] != self.state[i][j]:
                            nboard = Board(n)
                            nboard.state = copy.deepcopy(self.state)
                            nboard.state[new_x][new_y], nboard.state[i][j] = nboard.state[i][j], nboard.state[new_x][new_y]
                            new_states.append(nboard)
        return new_states

    def generate_states_bfs(self):
            n = self.size
            new_states = []
            
            for i in range(0, n):
                for j in range(0, n):
                    dir_x = [1,0,-1,0]
                    dir_y = [0,1,0,-1]
                    bad = False              
                    
                    for k in range(0, 4):
                        new_x = i+dir_x[k]
                        new_y = j+dir_y[k]
                        if new_x >=0 and new_x <self.size and new_y >=0 and new_y < self.size:
                            if self.state[new_x][new_y] != self.state[i][j]:
                                nboard = Board(n)
                                nboard.state = copy.deepcopy(self.state)
                                nboard.state[new_x][new_y], nboard.state[i][j] = nboard.state[i][j], nboard.state[new_x][new_y]
                                new_states.append(nboard)
            return new_states

## Search Alg

In [ ]:
########################
##  Search Algorithms ##
########################

## BFS Algorithm
@profile(store='bfs')
def bfs(cur_state, visited, itr):
    print('BFS Initial ', end='')
    cur_state.print_state()

    qu = []
    qu.append((cur_state, 0))
    parent = {}

    while(len(qu) != 0):
        itr[0]+=1
        cst, depth = qu.pop(0)
        print('')
        print('itr :',itr[0],' depth:',depth)
        
        cst.print_state()
        if cst.is_goal_state():
            itr[1] = depth
            return cst
        for state in cst.generate_states_bfs():
            y = repr(state.state)
            
            if y not in visited:
                visited.add(y)
                parent[y] = cst
                qu.append((state, depth+1))

    return None

def print_path(parents, goal, initial):
     print(parents)
     temp = goal
     while repr(temp.state) in parents.keys() and repr(temp.state) != repr(initial.state):
         temp.print_state()
         temp = parents[repr(temp.state)]
     temp.print_state()

## A* Algorithm
@profile(store='A*')
def a_star(cur_state, visited, itr):
    print('A* Initial ', end='')
    cur_state.print_state()

    hp = []
    heapq.heappush(hp, (0+cur_state.h(), 0, cur_state))
    parent = {}
    while(len(hp) != 0):
        itr[0]+=1
        top_ele = heapq.heappop(hp)
        cst = top_ele[2]
        #print('cst here',cst)
        if cst.is_goal_state():
            itr[1] = top_ele[1]
            print_path(parent, cst, cur_state)
            return cst
        cur_steps = top_ele[1]
        for state in cst.generate_states():
            y = repr(state.state)
            if y not in visited:
                visited.add(y)
                parent[y] = cst
                cost = cur_steps+1+state.h()
                heapq.heappush(hp, (cost, cur_steps+1, state))

    return None

#Board 3x3

###Initial Board

In [ ]:
board3x3 = Board(int(3))
board3x3.input_state(3)

Enter the colour Board state as 3x3 matrix, elements separated by a ",":
['\U0001f7e6', '\U0001f7e8', '\U0001f7e5']
['\U0001f7e8', '\U0001f7e8', '\U0001f7e5']
['\U0001f7e6', '\U0001f7e5', '\U0001f7e6']


###BFS Run

In [ ]:
#---BFS Run---
bfs(board3x3)

BFS Initial Board state: 
🟦🟨🟥
🟨🟨🟥
🟦🟥🟦

itr : 1  depth: 0
Board state: 
🟦🟨🟥
🟨🟨🟥
🟦🟥🟦

itr : 2  depth: 1
Board state: 
🟨🟨🟥
🟦🟨🟥
🟦🟥🟦

itr : 3  depth: 1
Board state: 
🟨🟦🟥
🟨🟨🟥
🟦🟥🟦

itr : 4  depth: 1
Board state: 
🟦🟥🟨
🟨🟨🟥
🟦🟥🟦

itr : 5  depth: 1
Board state: 
🟦🟨🟥
🟦🟨🟥
🟨🟥🟦

itr : 6  depth: 1
Board state: 
🟦🟨🟥
🟨🟥🟥
🟦🟨🟦

itr : 7  depth: 1
Board state: 
🟦🟨🟥
🟨🟥🟨
🟦🟥🟦

itr : 8  depth: 1
Board state: 
🟦🟨🟥
🟨🟨🟦
🟦🟥🟥

itr : 9  depth: 1
Board state: 
🟦🟨🟥
🟨🟨🟥
🟥🟦🟦

itr : 10  depth: 1
Board state: 
🟦🟨🟥
🟨🟨🟥
🟦🟦🟥

itr : 11  depth: 2
Board state: 
🟦🟨🟥
🟨🟨🟥
🟦🟥🟦

itr : 12  depth: 2
Board state: 
🟨🟥🟨
🟦🟨🟥
🟦🟥🟦

itr : 13  depth: 2
Board state: 
🟨🟨🟥
🟨🟦🟥
🟦🟥🟦

itr : 14  depth: 2
Board state: 
🟨🟨🟥
🟦🟥🟥
🟦🟨🟦

itr : 15  depth: 2
Board state: 
🟨🟨🟥
🟦🟥🟨
🟦🟥🟦

itr : 16  depth: 2
Board state: 
🟨🟨🟥
🟦🟨🟦
🟦🟥🟥

itr : 17  depth: 2
Board state: 
🟨🟨🟥
🟦🟨🟥
🟥🟦🟦

itr : 18  depth: 2
Board state: 
🟨🟨🟥
🟦🟨🟥
🟦🟦🟥

itr : 19  depth: 2
Board state: 
🟨🟥🟦
🟨🟨🟥
🟦🟥🟦

itr : 20  depth: 2
Board state: 
🟨🟦🟥
🟦🟨🟥
🟨🟥🟦

itr : 21  depth: 2
Board state: 
🟨🟦🟥
🟨🟥🟥
🟦🟨🟦

itr 

###3x3 A-star

In [ ]:
#---A star Run---
a_star(board3x3)

A* Initial Board state: 
🟦🟨🟥
🟨🟨🟥
🟦🟥🟦
{"[['\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e8'], ['\\U0001f7e8', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e6']]": <__main__.Board object at 0x7fc8547da5d0>, "[['\\U0001f7e8', '\\U0001f7e6', '\\U0001f7e5'], ['\\U0001f7e8', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e6']]": <__main__.Board object at 0x7fc8547da5d0>, "[['\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e8', '\\U0001f7e5', '\\U0001f7e6']]": <__main__.Board object at 0x7fc8547da5d0>, "[['\\U0001f7e8', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e6']]": <__main__.Board object at 0x7fc8547da5d0>, "[['\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e8', '\\U0001f7e5', '\\U0001f7e5'], ['\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e6']]": <__main__.Board object at 0x7fc8547da5d0>, "[['\\U0001f7e6', '\\U

In [ ]:

#print(json.dumps(stats, indent = 4))
jfile = json.dumps(stats, indent = 4)
print(jfile)

{
    "bfs": {
        "elapsed_time": 0.1135439800000313,
        "iters": 43,
        "path_length": 2,
        "cpu": 7.0
    },
    "A*": {
        "elapsed_time": 0.008087954999609792,
        "iters": 5,
        "path_length": 3,
        "cpu": 36.8
    }
}


#Board 4x4

In [ ]:
board4x4 = Board(int(4))
board4x4.input_state(4)

Enter the colour Board state as 4x4 matrix, elements separated by a ",":
['\U0001f7e5', '\U0001f7e6', '\U0001f7e5', '\U0001f7e6']
['\U0001f7e8', '\U0001f7e9', '\U0001f7e6', '\U0001f7e6']
['\U0001f7e9', '\U0001f7e9', '\U0001f7e8', '\U0001f7e5']
['\U0001f7e5', '\U0001f7e9', '\U0001f7e8', '\U0001f7e8']


In [ ]:
#---BFS Run---
bfs(board4x4)

Streaming output truncated to the last 5000 lines.
🟩🟩🟨🟥
🟥🟩🟨🟨

itr : 470  depth: 3
Board state: 
🟥🟥🟦🟩
🟨🟦🟦🟦
🟩🟩🟨🟥
🟥🟩🟨🟨

itr : 471  depth: 3
Board state: 
🟥🟥🟩🟦
🟩🟦🟦🟦
🟨🟩🟨🟥
🟥🟩🟨🟨

itr : 472  depth: 3
Board state: 
🟥🟥🟩🟦
🟦🟨🟦🟦
🟩🟩🟨🟥
🟥🟩🟨🟨

itr : 473  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟩🟦🟦
🟩🟦🟨🟥
🟥🟩🟨🟨

itr : 474  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟨🟦
🟩🟩🟦🟥
🟥🟩🟨🟨

itr : 475  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟥
🟩🟩🟨🟦
🟥🟩🟨🟨

itr : 476  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟦
🟥🟩🟨🟥
🟩🟩🟨🟨

itr : 477  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟦
🟩🟨🟩🟥
🟥🟩🟨🟨

itr : 478  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟦
🟩🟩🟥🟨
🟥🟩🟨🟨

itr : 479  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟦
🟩🟩🟨🟨
🟥🟩🟨🟥

itr : 480  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟦
🟩🟩🟨🟥
🟩🟥🟨🟨

itr : 481  depth: 3
Board state: 
🟥🟥🟩🟦
🟨🟦🟦🟦
🟩🟩🟨🟥
🟥🟨🟩🟨

itr : 482  depth: 3
Board state: 
🟥🟩🟦🟦
🟩🟦🟥🟦
🟨🟩🟨🟥
🟥🟩🟨🟨

itr : 483  depth: 3
Board state: 
🟥🟩🟦🟦
🟦🟨🟥🟦
🟩🟩🟨🟥
🟥🟩🟨🟨

itr : 484  depth: 3
Board state: 
🟥🟩🟦🟦
🟨🟩🟥🟦
🟩🟦🟨🟥
🟥🟩🟨🟨

itr : 485  depth: 3
Board state: 
🟥🟩🟦🟦
🟨🟦🟨🟦
🟩🟩🟥🟥
🟥🟩🟨🟨

itr : 486  depth: 3
Board state: 
🟥🟩🟦🟦
🟨🟦🟦🟥
🟩🟩🟨🟥
🟥🟩🟨🟨

itr

In [ ]:
#---A star Run---
a_star(board4x4)

A* Initial Board state: 
🟥🟦🟥🟦
🟨🟩🟦🟦
🟩🟩🟨🟥
🟥🟩🟨🟨
{"[['\\U0001f7e5', '\\U0001f7e6', '\\U0001f7e6', '\\U0001f7e5'], ['\\U0001f7e8', '\\U0001f7e9', '\\U0001f7e6', '\\U0001f7e6'], ['\\U0001f7e9', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e8']]": <__main__.Board object at 0x7fc854766f10>, "[['\\U0001f7e5', '\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e6'], ['\\U0001f7e8', '\\U0001f7e6', '\\U0001f7e9', '\\U0001f7e6'], ['\\U0001f7e9', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e8']]": <__main__.Board object at 0x7fc854766f10>, "[['\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e5', '\\U0001f7e6'], ['\\U0001f7e8', '\\U0001f7e6', '\\U0001f7e6', '\\U0001f7e6'], ['\\U0001f7e9', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e8']]": <__main__.Board object at 0x7fc854766f10>, "[['\\U0001f7e5', '\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e6'], ['\\U

In [ ]:
#print(json.dumps(stats, indent = 4))
jfile = json.dumps(stats, indent = 4)
print(jfile)

{
    "bfs": {
        "elapsed_time": 9.12026741200043,
        "iters": 1182,
        "path_length": 3,
        "cpu": 21.6
    },
    "A*": {
        "elapsed_time": 0.016341044000000693,
        "iters": 4,
        "path_length": 3,
        "cpu": 4.4
    }
}


#Board 5x5 (Run A star only)

In [ ]:
board5x5 = Board(int(5))
board5x5.input_state(5)

Enter the colour Board state as 5x5 matrix, elements separated by a ",":
['⬜️', '\U0001f7e6', '⬜️', '\U0001f7e9', '\U0001f7e6']
['⬜️', '⬜️', '\U0001f7e6', '\U0001f7e8', '\U0001f7e5']
['⬜️', '\U0001f7e6', '\U0001f7e5', '\U0001f7e6', '\U0001f7e9']
['\U0001f7e9', '\U0001f7e5', '\U0001f7e9', '\U0001f7e8', '\U0001f7e5']
['\U0001f7e8', '\U0001f7e8', '\U0001f7e5', '\U0001f7e9', '\U0001f7e8']


In [ ]:
#---BFS Run---
#bfs(board5x5)
#---A star Run---
a_star(board5x5)

A* Initial Board state: 
⬜️🟦⬜️🟩🟦
⬜️⬜️🟦🟨🟥
⬜️🟦🟥🟦🟩
🟩🟥🟩🟨🟥
🟨🟨🟥🟩🟨
{"[['\\U0001f7e6', '⬜️', '⬜️', '\\U0001f7e9', '\\U0001f7e6'], ['⬜️', '⬜️', '\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e5'], ['⬜️', '\\U0001f7e6', '\\U0001f7e5', '\\U0001f7e6', '\\U0001f7e9'], ['\\U0001f7e9', '\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e8', '\\U0001f7e8', '\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8']]": <__main__.Board object at 0x7fc853e49710>, "[['⬜️', '\\U0001f7e6', '⬜️', '\\U0001f7e9', '\\U0001f7e6'], ['⬜️', '\\U0001f7e6', '\\U0001f7e6', '\\U0001f7e8', '\\U0001f7e5'], ['⬜️', '⬜️', '\\U0001f7e5', '\\U0001f7e6', '\\U0001f7e9'], ['\\U0001f7e9', '\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8', '\\U0001f7e5'], ['\\U0001f7e8', '\\U0001f7e8', '\\U0001f7e5', '\\U0001f7e9', '\\U0001f7e8']]": <__main__.Board object at 0x7fc853e49710>, "[['⬜️', '\\U0001f7e6', '⬜️', '\\U0001f7e9', '\\U0001f7e6'], ['⬜️', '\\U0001f7e6', '⬜️', '\\U0001f7e8', '\\U0001f7e5'], ['⬜️', '\\U0001f7e6', '\\U0001f7e5', '\\U0001

In [ ]:
#print(json.dumps(stats, indent = 4))
jfile = json.dumps(stats, indent = 4)
print(jfile)

{
    "bfs": {
        "elapsed_time": 9.12026741200043,
        "iters": 1182,
        "path_length": 3,
        "cpu": 21.6
    },
    "A*": {
        "elapsed_time": 0.022159307000038098,
        "iters": 4,
        "path_length": 3,
        "cpu": 6.0
    }
}
